In [103]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import os
import joblib

In [104]:
ctl_samples = pd.read_csv('../data/control/control_samples.csv')
ctl_samples.shape

(5643, 2570)

In [105]:
cancer_samples = pd.read_csv('../data/cancer/prostate_cancer_samples.csv')
print(cancer_samples.shape)

(1027, 2570)


In [106]:
ctl_samples.columns = ctl_samples.columns.str.strip()
ctl_samples.head()

,Sample_ID,hsa-miR-28-3p,hsa-miR-27a-5p,hsa-miR-518b,hsa-miR-520b,hsa-miR-498,hsa-miR-512-3p,hsa-miR-491-5p,hsa-miR-490-3p,hsa-miR-452-5p,...,hsa-miR-6881-5p,hsa-miR-6880-3p,hsa-miR-6873-5p,hsa-miR-6872-3p,hsa-miR-6865-5p,hsa-miR-6864-3p,Sex,Age,Stage,Disease
0,CHC672,-3.396181,-3.396181,1.347562,-3.396181,5.840556,0.486324,5.268448,2.210799,-3.396181,...,1.639051,5.446718,-3.396181,5.004431,4.415208,2.881128,Sex: Male,age: 79,Stage: NA,disease state: no cancer
1,CHC673,0.821318,2.857542,3.670470,-2.074762,6.143286,2.378031,6.037746,3.703771,1.438320,...,1.731415,6.223110,-2.074762,5.536943,6.122610,4.933096,Sex: Male,age: 66,Stage: NA,disease state: no cancer
2,CHC681,2.180237,1.718978,4.354315,4.258323,6.214003,-3.339112,5.673198,2.663074,0.651702,...,2.271011,5.580478,-3.339112,4.750161,5.703635,2.825348,Sex: Male,age: 65,Stage: NA,disease state: no cancer
3,CHC684,-2.203483,-2.203483,3.419279,-2.203483,6.015818,-2.203483,6.504094,-2.203483,1.275758,...,2.470047,5.677397,-2.203483,4.918763,5.480697,-2.203483,Sex: Male,age: 61,Stage: NA,disease state: no cancer
4,CHC688,-3.558819,-3.558819,2.783801,0.414324,6.077240,-3.558819,5.821377,3.112312,-1.533649,...,3.052431,5.311382,1.195303,4.862444,4.913932,3.920142,Sex: Female,age: 72,Stage: NA,disease state: no cancer


In [107]:
ctl_samples['Stage'] = ctl_samples['Stage'].str.replace('Stage: NA', 'Stage: CTL')

# Verify the replacements
print(ctl_samples['Stage'])

0       Stage: CTL
1       Stage: CTL
2       Stage: CTL
3       Stage: CTL
4       Stage: CTL
           ...    
5638    Stage: CTL
5639    Stage: CTL
5640    Stage: CTL
5641    Stage: CTL
5642    Stage: CTL
Name: Stage, Length: 5643, dtype: object


In [108]:
combined_dataset = pd.concat([cancer_samples, ctl_samples], ignore_index=True)

In [109]:
# Verify and clean the Stage column
print("Unique values in Stage column:", combined_dataset['Stage'].unique())
combined_dataset['Stage'] = combined_dataset['Stage'].str.strip()
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] == 'Stage: CTL', 0, 1)
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] != 'Stage: CTL', 1, combined_dataset['ID_REF'])

# Print class distribution to ensure both classes are present
print("Class distribution in ID_REF column:")
print(combined_dataset['ID_REF'].value_counts())

Unique values in Stage column: ['Stage: 3' 'Stage: 2' 'Stage: NA' 'Stage: 4' 'Stage: 1' 'Stage: CTL']
Class distribution in ID_REF column:
ID_REF
0    5643
1    1027
Name: count, dtype: int64


In [110]:
def process_data(data, under_sample_factor=None, over_sample_factor=None):
    columns_to_drop = ['Sample_ID', 'Sex', 'Age', 'Stage', 'Disease']
    data = data.drop(columns=columns_to_drop, axis=1)
    
    # Ensure ID_REF is handled separately
    id_ref = data["ID_REF"]
    data = data.drop(["ID_REF"], axis=1)
    
    # Convert all remaining columns to numeric, coercing errors to NaN
    data = data.apply(pd.to_numeric, errors='coerce')
    
    # Handle NaN values, e.g., fill with the mean of each column
    data = data.fillna(data.mean())
    
    # Combine ID_REF back to the DataFrame
    data["ID_REF"] = id_ref

    # Convert to numpy arrays
    x = np.array(data.drop(["ID_REF"], axis=1)).astype('float')
    y = np.array(data["ID_REF"]).astype('int')
    feature_names = data.columns[:-1]

    if under_sample_factor is not None and isinstance(under_sample_factor, float) and 0 < under_sample_factor <= 1:
        under_sampler = RandomUnderSampler(sampling_strategy=under_sample_factor)
        x, y = under_sampler.fit_resample(x, y)

    if over_sample_factor is not None and isinstance(over_sample_factor, float) and 0 < over_sample_factor <= 1:
        over_sampler = RandomOverSampler(sampling_strategy=over_sample_factor)
        x, y = over_sampler.fit_resample(x, y)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test, feature_names

In [111]:
# Define parameters
feature_selection_num = 50
feature_importance_num = 10

num_cancer_samples = combined_dataset['ID_REF'].value_counts()[1]  # Number of benign samples
num_control_samples = combined_dataset['ID_REF'].value_counts()[0]  # Number of control samples
under_sample_factor = num_cancer_samples / num_control_samples  # Factor to balance the classes

# Process the data with under-sampling
x_train, x_test, y_train, y_test, feature_names = process_data(combined_dataset, under_sample_factor=under_sample_factor, over_sample_factor=None)

/var/folders/c_/rs13jmts3nbf9xfm917mh9p80000gn/T/ipykernel_77761/2988405077.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ID_REF"] = id_ref


In [112]:
def svm_objective(trial):
    k = feature_selection_num
    
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    kernel = 'linear'  # Use only linear kernel for feature extraction
    
    # Relaxed LassoCV with a wider range of smaller alphas
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', SVC(C=C, kernel=kernel, random_state=0))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    try:
        scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
        return scores.mean()
    except ValueError as e:
        print(f"ValueError: {e}")
        print("Selected features by SelectKBest:", skb.get_support(indices=True))
        if hasattr(lasso.estimator_, 'coef_'):
            print("Lasso coefficients:", lasso.estimator_.coef_)
        return np.nan

In [113]:
def rf_objective(trial):
    k = feature_selection_num
    
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, criterion=criterion, random_state=0))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [114]:
def xgboost_objective(trial):
    k = feature_selection_num
    
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1.0, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    
    lasso = SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))
    
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('lasso', lasso),
        ('estimator', XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, random_state=0, use_label_encoder=False, eval_metric='logloss'))
    ])
    
    cv = StratifiedKFold(n_splits=5)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [115]:
# Function to save study
def save_study(study, filename):
    joblib.dump(study, filename)

# Function to load study
def load_study(filename):
    return joblib.load(filename)

In [116]:
# Optimize hyperparameters using Optuna
svm_study_filename = '50_svm_study_ctl_c.pkl'
rf_study_filename = '50_rf_study_ctl_c.pkl'
xgboost_study_filename = '50_xgboost_study_ctl_c.pkl'

In [117]:
# Optimize hyperparameters using Optuna with early stopping
def optimize_with_early_stopping(objective, study_filename, n_trials=50, patience=10):
    if os.path.exists(study_filename):
        study = load_study(study_filename)
        return study
    else:
        study = optuna.create_study(direction='maximize')
    
    best_value = -np.inf
    trials_without_improvement = 0
    
    for trial in range(n_trials):
        study.optimize(objective, n_trials=1)
        
        current_best_value = study.best_value
        if current_best_value > best_value:
            best_value = current_best_value
            trials_without_improvement = 0
        else:
            trials_without_improvement += 1
        
        if trials_without_improvement >= patience:
            print(f"Early stopping at trial {trial + 1}")
            break
        
        save_study(study, study_filename)
    
    return study

### Find Hyperparmeters if not trained already

In [118]:
svm_study = optimize_with_early_stopping(svm_objective, svm_study_filename, n_trials=50, patience=10)

In [119]:
rf_study = optimize_with_early_stopping(rf_objective, rf_study_filename, n_trials=100, patience=10)

In [120]:
xgboost_study = optimize_with_early_stopping(xgboost_objective, xgboost_study_filename, n_trials=100, patience=10)

### Print the best trial for each study

For SVM, RF, and XGBoost

In [121]:
print("Best SVM trial:")
svm_trial = svm_study.best_trial
print("  Value: ", svm_trial.value)
print("  Params: ")
for key, value in svm_trial.params.items():
    print(f"    {key}: {value}")

Best SVM trial:
  Value:  0.9996251171508904
  Params: 
    C: 10.832689243478363


In [122]:
print("Best Random Forest trial:")
rf_trial = rf_study.best_trial
print("  Value: ", rf_trial.value)
print("  Params: ")
for key, value in rf_trial.params.items():
    print(f"    {key}: {value}")

Best Random Forest trial:
  Value:  0.9996251171508904
  Params: 
    n_estimators: 272
    max_depth: 26
    max_features: sqrt
    criterion: gini


In [123]:
print("Best XGBoost trial:")
xgboost_trial = xgboost_study.best_trial
print("  Value: ", xgboost_trial.value)
print("  Params: ")
for key, value in xgboost_trial.params.items():
    print(f"    {key}: {value}")

Best XGBoost trial:
  Value:  0.9996251171508904
  Params: 
    learning_rate: 0.01828145650307612
    max_depth: 6
    n_estimators: 917


## Train and evaluate the models with the best hyperparameters

In [124]:
# Train and evaluate the models with the best hyperparameters
def train_and_evaluate(pipe, x_train, y_train, x_test, y_test):
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    print(f'Testing accuracy {accuracy_score(y_test, y_pred)}')
    print(f'Confusion matrix: \n{confusion_matrix(y_test, y_pred)}')

### SVM

In [125]:
best_svm_params = svm_trial.params
svm_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))),
    ('estimator', SVC(C=best_svm_params['C'], kernel='linear', random_state=0))  # Ensure linear kernel
])
train_and_evaluate(svm_pipe, x_train, y_train, x_test, y_test)

Testing accuracy 0.9992503748125937
Confusion matrix: 
[[1128    1]
 [   0  205]]


In [126]:
# svm_accuracies = []

# # Loop through the number of features from 1 to 150
# for k in range(1, 121):
#     best_svm_params = svm_trial.params
#     svm_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', SVC(C=best_svm_params['C'], kernel=best_svm_params['kernel'], random_state=0))
#     ])
    
#     # Train and evaluate the pipeline
#     svm_pipe.fit(x_train, y_train)
#     y_pred = svm_pipe.predict(x_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     svm_accuracies.append(accuracy)
#     print(f'Number of features: {k}, Accuracy: {accuracy}')

# # Plot the number of features vs accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), svm_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features (SVM)')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

### Random Forest

In [127]:
best_rf_params = rf_trial.params
rf_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0))),
    ('estimator', RandomForestClassifier(n_estimators=best_rf_params['n_estimators'],
                                         max_depth=best_rf_params['max_depth'],
                                         max_features=best_rf_params['max_features'],
                                         criterion=best_rf_params['criterion'],
                                         random_state=0))
])
train_and_evaluate(rf_pipe, x_train, y_train, x_test, y_test)

Testing accuracy 0.9992503748125937
Confusion matrix: 
[[1128    1]
 [   0  205]]


In [128]:
# rf_accuracies = []

# # Loop through the number of features from 1 to 200
# for k in range(1, 121):
#     best_rf_params = rf_trial.params
#     rf_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', RandomForestClassifier(n_estimators=best_rf_params['n_estimators'],
#                                              max_depth=best_rf_params['max_depth'],
#                                              max_features=best_rf_params['max_features'],
#                                              criterion=best_rf_params['criterion'],
#                                              random_state=0))
#     ])
    
#     # Train and evaluate the pipeline
#     rf_pipe.fit(x_train, y_train)
#     y_pred = rf_pipe.predict(x_test)
#     rf_accuracy = accuracy_score(y_test, y_pred)
#     rf_accuracies.append(rf_accuracy)
#     print(f'Number of features: {k}, Accuracy: {rf_accuracy}')

# # Plot the number of features vs rf_accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), rf_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

### XGBoost

In [129]:
best_xgboost_params = xgboost_trial.params
xgboost_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
    ('estimator', XGBClassifier(learning_rate=best_xgboost_params['learning_rate'],
                                max_depth=best_xgboost_params['max_depth'],
                                n_estimators=best_xgboost_params['n_estimators'],
                                random_state=0,
                                use_label_encoder=False,
                                eval_metric='logloss'))
])
train_and_evaluate(xgboost_pipe, x_train, y_train, x_test, y_test)

/Users/emmetthintz/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.838e-01, tolerance: 6.954e-02
  model = cd_fast.enet_coordinate_descent(


Testing accuracy 1.0
Confusion matrix: 
[[1129    0]
 [   0  205]]


In [130]:
# xgboost_accuracies = []

# # Loop through the number of features from 1 to 150
# for k in range(1, 121):
#     best_xgboost_params = xgboost_trial.params
#     xgboost_pipe = Pipeline([
#         ('skb', SelectKBest(f_classif, k=k)),
#         ('lasso', SelectFromModel(LassoCV(cv=5, random_state=0))),
#         ('estimator', XGBClassifier(learning_rate=best_xgboost_params['learning_rate'],
#                                     max_depth=best_xgboost_params['max_depth'],
#                                     n_estimators=best_xgboost_params['n_estimators'],
#                                     random_state=0,
#                                     use_label_encoder=False,
#                                     eval_metric='logloss'))
#     ])

#     # Train and evaluate the pipeline
#     xgboost_pipe.fit(x_train, y_train)
#     y_pred = xgboost_pipe.predict(x_test)
#     xgboost_accuracy = accuracy_score(y_test, y_pred)
#     xgboost_accuracies.append(xgboost_accuracy)
#     print(f'Number of features: {k}, Accuracy: {xgboost_accuracy}') 

# # Plot the number of features vs xgboost_accuracy
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, 121), xgboost_accuracies, marker='o')
# plt.title('Accuracy vs Number of Features (XGBoost)')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.grid(True)
# plt.show()

In [131]:
def get_top_features(pipe, feature_names, top_feature_num):
    if isinstance(pipe.named_steps['estimator'], SVC):
        if pipe.named_steps['estimator'].kernel != 'linear':
            raise ValueError("Feature importance is not available for non-linear SVM kernels.")
        feature_scores = pipe.named_steps['estimator'].coef_[0]
    elif isinstance(pipe.named_steps['estimator'], XGBClassifier):
        feature_scores = pipe.named_steps['estimator'].feature_importances_
    else:
        raise ValueError("Unsupported estimator type for feature extraction.")
    
    # Get selected features from SelectKBest
    skb_support = pipe.named_steps['skb'].get_support(indices=True)
    print("Selected features from SelectKBest:", skb_support)
    
    # Transform the features using SelectKBest
    skb_features = pipe.named_steps['skb'].transform(x_train)
    
    # Get selected features from LassoCV
    lasso = LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0)
    lasso.fit(skb_features, y_train)
    lasso_support = np.where(lasso.coef_ != 0)[0]
    print("Selected features from LassoCV:", lasso_support)
    
    if len(lasso_support) == 0:
        print("No features selected after LassoCV.")
        return []
    
    # Ensure indices match feature_scores
    top_indices = np.argsort(np.abs(lasso.coef_[lasso_support]))[::-1][:top_feature_num]
    top_features = [(feature_names[skb_support[i]], lasso.coef_[lasso_support[i]]) for i in top_indices]
    
    return top_features

In [132]:
try:
    svm_top_features = get_top_features(svm_pipe, feature_names, feature_importance_num)
    print("Top SVM features:", svm_top_features)
except ValueError as e:
    print("SVM feature extraction error:", e)

xgboost_top_features = get_top_features(xgboost_pipe, feature_names, feature_importance_num)
print("Top XGBoost features:", xgboost_top_features)

Selected features from SelectKBest: [  76  153  221  236  388  423  463  590  697  726  739  780  785  821
  832  837  905  931  947 1044 1109 1171 1174 1187 1214 1254 1273 1361
 1386 1483 1493 1511 1560 1562 1616 1724 1754 1781 1812 2058 2131 2195
 2214 2227 2285 2294 2309 2386 2460 2473]
Selected features from LassoCV: [ 0  1  2  3  4  5  6  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 30 31 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49]
Top SVM features: [('hsa-miR-6746-5p', -0.07996244612460844), ('hsa-miR-642b-3p', 0.05079726486966696), ('hsa-miR-663a', 0.0391250467252818), ('hsa-miR-3917', 0.0364765313137464), ('hsa-miR-8069', -0.036189824095779186), ('hsa-miR-6766-5p', -0.029477473841805397), ('hsa-miR-1247-3p', 0.028917759671351295), ('hsa-miR-4690-5p', -0.02697001353848485), ('hsa-miR-92a-2-5p', 0.026438922793997722), ('hsa-miR-3940-5p', -0.018227966270909154)]
Selected features from SelectKBest: [  76  153  221  236  388  423  463  590  697  726  739  780  785  

In [133]:
def normalize_importance(importance_scores):
    max_score = max(importance_scores, key=lambda x: abs(x[1]))[1]
    min_score = min(importance_scores, key=lambda x: abs(x[1]))[1]
    normalized_scores = [(feature, (score - min_score) / (max_score - min_score)) for feature, score in importance_scores]
    return normalized_scores

In [134]:
# Normalize the top features for SVM and XGBoost
normalized_svm_features = normalize_importance(svm_top_features)
normalized_xgboost_features = normalize_importance(xgboost_top_features)

# Compile normalized top features
def compile_normalized_top_features(normalized_svm_features, normalized_xgboost_features):
    top_features = []
    for feature, score in normalized_svm_features:
        top_features.append((f'SVM_{feature}', score))
    for feature, score in normalized_xgboost_features:
        top_features.append((f'XGB_{feature}', score))
    return top_features

normalized_top_features = compile_normalized_top_features(normalized_svm_features, normalized_xgboost_features)
print("Compiled normalized top features:", normalized_top_features)

Compiled normalized top features: [('SVM_hsa-miR-6746-5p', 1.0), ('SVM_hsa-miR-642b-3p', -1.1180985294466679), ('SVM_hsa-miR-663a', -0.929027232951639), ('SVM_hsa-miR-3917', -0.8861255122631041), ('SVM_hsa-miR-8069', 0.2909534164284971), ('SVM_hsa-miR-6766-5p', 0.1822240601614487), ('SVM_hsa-miR-1247-3p', -0.7636854810146321), ('SVM_hsa-miR-4690-5p', 0.14160720699830845), ('SVM_hsa-miR-92a-2-5p', -0.7235322816481189), ('SVM_hsa-miR-3940-5p', -0.0), ('XGB_hsa-miR-6746-5p', 1.0), ('XGB_hsa-miR-642b-3p', -1.1180985294466679), ('XGB_hsa-miR-663a', -0.929027232951639), ('XGB_hsa-miR-3917', -0.8861255122631041), ('XGB_hsa-miR-8069', 0.2909534164284971), ('XGB_hsa-miR-6766-5p', 0.1822240601614487), ('XGB_hsa-miR-1247-3p', -0.7636854810146321), ('XGB_hsa-miR-4690-5p', 0.14160720699830845), ('XGB_hsa-miR-92a-2-5p', -0.7235322816481189), ('XGB_hsa-miR-3940-5p', -0.0)]


In [135]:
normalized_top_features_df = pd.DataFrame(normalized_top_features, columns=['Feature', 'Normalized Importance'])
normalized_top_features_df.to_csv('../GSEA/miRNA/ctl_c_miRNA.csv', index=False)
print("Compiled normalized top features saved to '../GSEA/miRNA/ctl_c_miRNA.csv'")

Compiled normalized top features saved to '../GSEA/miRNA/ctl_c_miRNA.csv'


In [136]:
def get_all_selected_features(pipe, feature_names, classifier_name):
    if isinstance(pipe.named_steps['estimator'], SVC):
        if pipe.named_steps['estimator'].kernel != 'linear':
            raise ValueError("Feature importance is not available for non-linear SVM kernels.")
        feature_scores = pipe.named_steps['estimator'].coef_[0]
    elif isinstance(pipe.named_steps['estimator'], XGBClassifier):
        feature_scores = pipe.named_steps['estimator'].feature_importances_
    else:
        raise ValueError("Unsupported estimator type for feature extraction.")
    
    # Get selected features from SelectKBest
    skb_support = pipe.named_steps['skb'].get_support(indices=True)
    print("Selected features from SelectKBest:", skb_support)
    
    # Transform the features using SelectKBest
    skb_features = pipe.named_steps['skb'].transform(x_train)
    
    # Get selected features from LassoCV
    lasso = LassoCV(alphas=np.logspace(-6, -1, 50), cv=5, max_iter=10000, random_state=0)
    lasso.fit(skb_features, y_train)
    lasso_support = np.where(lasso.coef_ != 0)[0]
    print("Selected features from LassoCV:", lasso_support)
    
    # Map the selected LassoCV features back to the original feature names
    lasso_support_mapped = skb_support[lasso_support]
    
    # Ensure the index does not go out of bounds
    all_features = [(feature_names[i], feature_scores[j], classifier_name) for j, i in enumerate(lasso_support_mapped) if j < len(feature_scores)]
    
    return all_features

In [137]:
# Extract all selected features from SVM and XGBoost
all_svm_features = get_all_selected_features(svm_pipe, feature_names, 'SVM')
print("All SVM features:", all_svm_features)

all_xgboost_features = get_all_selected_features(xgboost_pipe, feature_names, 'XGBoost')
print("All XGBoost features:", all_xgboost_features)

# Combine the features and save to CSV
all_features_df = pd.DataFrame(all_svm_features + all_xgboost_features, columns=['Feature', 'Importance', 'Classifier'])
all_features_df.to_csv('../GSEA/miRNA/50_ctl_c.csv', index=False)
print("All selected features saved to '../GSEA/miRNA/50_ctl_c.csv'")

Selected features from SelectKBest: [  76  153  221  236  388  423  463  590  697  726  739  780  785  821
  832  837  905  931  947 1044 1109 1171 1174 1187 1214 1254 1273 1361
 1386 1483 1493 1511 1560 1562 1616 1724 1754 1781 1812 2058 2131 2195
 2214 2227 2285 2294 2309 2386 2460 2473]
Selected features from LassoCV: [ 0  1  2  3  4  5  6  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 30 31 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49]
All SVM features: [('hsa-miR-8073', 0.1569028212563373, 'SVM'), ('hsa-miR-6861-5p', -0.11564296147886538, 'SVM'), ('hsa-miR-5100', 0.029176992293050236, 'SVM'), ('hsa-miR-8069', -0.16387717170145616, 'SVM'), ('hsa-miR-6802-5p', -2.6407980588486332e-05, 'SVM'), ('hsa-miR-1307-3p', 0.11646436380282953, 'SVM'), ('hsa-miR-6729-5p', 0.13430249137359107, 'SVM'), ('hsa-miR-6131', 0.20900271643512902, 'SVM'), ('hsa-miR-320a', -0.04102615483198485, 'SVM'), ('hsa-miR-614', 0.08277348089859594, 'SVM'), ('hsa-miR-1247-3p', -0.006702252832956896, 'S